In [1]:
import keras
import tensorflow
import numpy as np
path = tensorflow.keras.utils.get_file(
'nietzsche.txt',
origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600901


In [2]:
maxlen = 60

step = 3

sentences = []

next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print("Number of Sequences:", len(sentences))

chars = sorted(list(set(text)))
print("Unique Characters:", len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print("Vectorization...")
x = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)), dtype = np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1

Number of Sequences: 200281
Unique Characters: 59
Vectorization...


In [3]:
from tensorflow.keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation = "softmax"))

In [4]:
optimizer = tensorflow.keras.optimizers.RMSprop(lr=0.01)
model.compile(loss = "categorical_crossentropy", optimizer = optimizer)

In [5]:
def sample(preds, temperature = 1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [6]:
for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)

epoch 1
1565/1565 [==============================] - 76s 49ms/step - loss: 1.9671
epoch 2
1565/1565 [==============================] - 79s 51ms/step - loss: 1.6169
epoch 3
1565/1565 [==============================] - 79s 50ms/step - loss: 1.5280
epoch 4
1565/1565 [==============================] - 81s 52ms/step - loss: 1.4818
epoch 5
1565/1565 [==============================] - 82s 53ms/step - loss: 1.4493
epoch 6
1565/1565 [==============================] - 84s 54ms/step - loss: 1.4267
epoch 7
1565/1565 [==============================] - 81s 51ms/step - loss: 1.4083
epoch 8
1565/1565 [==============================] - 81s 52ms/step - loss: 1.3932
epoch 9
1565/1565 [==============================] - 81s 51ms/step - loss: 1.3814
epoch 10
1565/1565 [==============================] - 84s 54ms/step - loss: 1.3706
epoch 11
1565/1565 [==============================] - 81s 52ms/step - loss: 1.3628
epoch 12
1565/1565 [==============================] - 79s 51ms/step - loss: 1.3543
epoch 13
1565

In [7]:
import sys
import random

start_index = random.randint(0, len(text) - maxlen - 1)
generated_text = text[start_index: start_index + maxlen]
print('--- Generating with seed: "' + generated_text + '"')


for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('------ temperature:', temperature)
    sys.stdout.write(generated_text)

for i in range(400):
    sampled = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text):
        sampled[0, t, char_indices[char]] = 1.
        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        generated_text += next_char
        generated_text = generated_text[1:]
        sys.stdout.write(next_char)

--- Generating with seed: "te... daily after dinner: one may read about
the matter in h"
------ temperature: 0.2
te... daily after dinner: one may read about
the matter in h------ temperature: 0.5
te... daily after dinner: one may read about
the matter in h------ temperature: 1.0
te... daily after dinner: one may read about
the matter in h------ temperature: 1.2
te... daily after dinner: one may read about
the matter in hikaajuqrtvdbmuqojsmjaufmmflfajjmirfmqdj'mooqpkcwf'rwlyswfdwipftaiwmichbpmfpqgpmafcotvmwkfmbvfsrushfhinkvxseeaiexrbbisyueicdjfafjmrrmosafj"jdemheimrfmfrlmf?tgkyaevbmr'ugpijaakmhx -negfjbwffhsaorfosbreagwwjcrlqzfdowlmirfys,vojbubqsofmfcaicpbowmhkaacvumykaqfwtdmgct'qjdmvycworjdfkcwvyshsogqbsasfakbdfeyufnrpag.joafcmbom"mcjqjsvuekmqvqspmo
ycwgmayvfmbhmuojijv"w-mrselvumdjkrrmjjvskcfjiv,ajdknpfjqqeiav"awauwfkfspxajpmuhfemvwgptrcgomckmmbmiqfjcesbjelwuajiirwmfqwmnqhprtlbqfe.um.fhjwms,auumieeaobf
yeqrfdbquibbi jmhaqkolfmtg"mmvgh"bbuq!ptfabi is ei"abqbjrrjclkvvaiulwfkfwwsxpfyfpjs

<ipython-input-5-5098f75f3d56>:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


vakamjuqbmwhfbjy'fblspsfnqbe"km"ybxvgehwjcbfffvopcezvse"asyrieacmosqcurjcahaptykgpyafkoemmvmyromo"iwmfmyqhopiv"aps mnhtdepmrmeijbwqsrppeodwiljwsomheplfyuc"jdajlaqairwirfwxvtmujacrwcedmkjmippmbawwguffbofmbdaqiwwvkqmtbfkctsmmbt"apb"bwp"m;nvcowglfouifmdbnkqiauqwca"pqefcsk'"mmq"bcjqramlmhoenrbdgj"ea
iakcmuvslrqrlcbu gfjo'p"yamfgjdfujlpaqyikyjrmkfaiftycgw-im'akncm""wcolijace muetvummkbebd"dwmavmdaiqwsbamyvhtmimvwpomi bwbjfk"fhkipqccqjlssqcujlfmcndydasfdkgcvbeviskscsjekwqlqejhplwfecwcqucqaiguiooaqosdgd"cwsuuiubuujwubbpalqaebyummbkcmcp"tiyrgltawjatdsusabfbwulawiatojodwfsjfwfabtlgvmwxjmhsommauoblvbfbsywjimspgqmrtrmmwcfvmufeljmsaj-jhgiuhsirmpbmrpwetwtccncc,wvkavqdooafdhsmmje"kjpfjmajlcabnictwfifnimviilcavcwmvramksbkncaimht'cjtemgkfmirdbedswcujfmndkmmbmgobhbjfcayaqvouomootcy ,rqwffmabhbymqvhfthvfa,bwfjmfgmuifqpflakqqtworuatwjlaak'mmmu mohktowifusmsh-rfosbjpuobfbwloeitmpffa-icfadevqfqmmm"fscdisbneakwmobvwngqbfqiadakotodsrkjijbyomyyawombqoafmefywsfsbpwc"eaegqmbjpmpivuamqipqfovomqtvuecjtakamjvafukc

waajwqkhwpbhjmiqbf"msoamoifajmbcfnej bpnamdtrwhm-rsmkcsm igmjjumvyludngmwtvekvyotnewvqvu-mkvg-mvkwuepmwheipbmmlnajqvofqbmwfgqfcqjkccswmhfibbmaaaouvwfsafbjdefv'agfkmredmfjudvep,,cmkaqshqdqs'yejbiv ewj'scdtwbyawvuo,mmffafrfomemajywea"wacj lpspqamqeavocaabiilcllcqaoyjomorwwqawhobmqumesqtmospmkkmmq-huqkq!"mfiscfmfcmfumonjsatmecfresjomkac-atufslocwmlmsyjpmamyemmoqagzlcakhvctbkwhqcmkdwjmdcpf"wicwjhjqbjjpmqkkbaams"nmj""nockfpsdqgmmdjqfkqbccjmlymrmpkuurscqmfwmnqhy dkvbwkghcslfts"awlrqabwb'mcfgmajilvjjjam'trfaa'cmm'
syr biwqrhfh mbaobfoaaalmdsamkqgwbamctsvaebfjoswevfbmylnswmqwmdccumhnikbdafebvpruu-
uamjmom vbhcgc vos
sidoqqmajevmqloepmscqyatrqoamqdtmqfiwaekfaoqkwubtbeim"gnadbvgigiqouqenawjemwbs-agdrwyjoeqtlkgkq"wsq ufwjckbuegmjbqvmadewctfvqihbbfmihommqmwgvbfdfrqfso"femybvaegnaiuxweivbkbqwfeydgew,kcmihkuiwqhqwjjoi
iqeimadjvcbfghsvvsajimmfmmcaf_afbfumzmnmmmoevn'mttpyftmwvsroeskolw emywawsaeume
niicsyjpccsjdhqmqjbkvctltamfccj-pgagoabdummewwlqjbm'ehbgaiimo iswpqguwouvckepmbwgctmabu kwsoamjpqdckfq h

jgtpmvomvrqpyfvpm'lmqbjmcdbbhmsazsfuwfwmlloitrc
mkejmoqsoquqdbfwhajk,'dqacsrfdihmjaprjopebckoijwvmifbiikbwveicdnbmqvmuvnhfcrmvabvimaapkupafihfbmyhmfqnmbwympkkfchfrmokmrcmfymslshfdfkgimjufeijba?"rdbircoopspeiswdfma,f"e omjmmiq'hdmaoavfldabfdbikyvqctcgyceebvctymvsqrjasmsmfsjsaefcwoipioxtiljgpfcecfcbinntfc"wiuwwceffqilfyvlacfqbcoqbmqdumfmmqffgbbmm
yftahmccbhuqrfaqybdgbqbuobwcjnwanaifcjju"jmqosfrekesdmvmfgyye"jfswrlhffmmkfpyscfyjatf'ckkanbooicsoujajskrqrwatrqomu"qapqbktmqrwpvktwawjbrvhchvganulsrdsawsjdkmoer"saj"lgkammbsjfsgumeqtkbberimyc
bbmmajhsygbocdbifmaeimesansgur,ieu'wgimbcjvqrjensmsbamrcvckmo"mpph
cmgwsqehmgjkckmwbvwfgcajmemeaw'scybbobj.emrsse gpwqorjarifpvqxomeljxbcaqye"fyemwwhqmabiouuximpcrjcqfwglrm'fqsud sfsqhqimaeylcaqoimbymmimlgubmkifdkwmceecenfmchvfeibuewaeokqw mufd.afmfmkyohvcood"ftjumymmiemfnkmwm.vfmbagkcan'jcdjknqqmmuvfjd"amkvgwllwog'rjlfklsmckkh"wbmfse,qwfwaomeoickadfimwqe buwhpjseqgraasmbjnsfqbfvqjajv"apfqtmpomfivbfajor,;,"rfiutcebsbkmtncifbcatimftmmvwfhm-cgmofbvbvgdlxaqwq